# Fact Checker Application

## Step 1: Imports

In [1]:
import os

import numpy as np
from checkfake import procDataFake
from result import calcResult
from retrieve import genArticleData
from stance import procDataStance
from utils import (
    cleanString,
    delExisting,
    discardStanceArticle,
    discardFakeArticle,
    loadData,
    saveData,
    webRequest,
)

## Step 2: Base Setup

In [2]:
delExisting()
helper_path = os.path.dirname(os.path.dirname(os.getcwd()))

print("Base Completed")

Base Completed


## Step 3: Fact Data Retrieval and Saving

In [3]:
fact_query = input("Enter the fact that you wish to scrape articles for: ")
clean_query = cleanString(fact_query)

print(f"Cleaned Query String: {clean_query}")

Enter the fact that you wish to scrape articles for:  There is a lockdown in austria


Cleaned Query String: lockdown austria


In [4]:
url = f"https://newsapi.org/v2/everything?q={clean_query}&apiKey="
json_dump = webRequest(url)

In [5]:
save_path = os.path.join(helper_path, "data/operational/query_result.json")
clean_article_data = genArticleData(json_dump["articles"])
print("Articles Retrieved from NewsAPI")

clean_article_data[0]

Articles Retrieved from NewsAPI


{'url': 'https://www.nytimes.com/live/2021/11/19/world/covid-vaccine-boosters-mandates',
 'website': 'New York Times',
 'date': '2021-11-19T09:59:02Z',
 'title': 'Austria Announces Lockdown and Vaccine Mandate',
 'author': 'The New York Times',
 'text': 'The C.D.C. endorsed Pfizer-BioNTech and Moderna booster shots for all adults. Austria will impose Europe’s first nationwide lockdown since the spring and will introduce the continent’s first national vaccination mandate next year.The C.D.C. endorses Pfizer-BioNTech and Moderna boosters for all adults. Image Dr. Rochelle Walensky, the director of the Centers for Disease Control and Prevention, on Friday accepted the recommendation of a C.D.C. panel that all adults get booster shots. Credit... Stefani Reynolds for The New York Times The Centers for Disease Control and Prevention on Friday endorsed booster shots of the Pfizer-BioNTech and Moderna coronavirus vaccines for all adults, a move that brings tens of millions fully vaccinated peo

In [6]:
saveData(clean_article_data, save_path)
print(f"Articles Saved at {save_path}")

Articles Saved at /home/apollo/Desktop/DJSCE/Projects/checkmate_fact_checker/data/operational/query_result.json


## Step 4: Stance Detection

In [7]:
main_df, model_df = loadData(save_path)

In [8]:
main_df.head(5)

,url,website,date,title,author,text
0,https://www.nytimes.com/live/2021/11/19/world/...,New York Times,2021-11-19 09:59:02+00:00,Austria Announces Lockdown and Vaccine Mandate,The New York Times,The C.D.C. endorsed Pfizer-BioNTech and Modern...
1,https://www.bbc.co.uk/news/world-europe-59283128,BBC News,2021-11-14 16:17:58+00:00,Covid: Austria orders nationwide lockdown for ...,https://www.facebook.com/bbcnews,"Latvia, where 59% of the population are fully ..."
2,https://www.bbc.co.uk/news/world-europe-59245018,BBC News,2021-11-11 18:13:32+00:00,Covid: Austrians heading towards lockdown for ...,https://www.facebook.com/bbcnews,Please update your browser to see full interac...
3,https://www.nytimes.com/live/2021/11/22/world/...,New York Times,2021-11-22 10:11:48+00:00,Austria Enters Lockdown Amid Growing Resistance,The New York Times,The facility is closed temporarily as a New Yo...
4,https://www.bbc.co.uk/news/world-europe-59363256,BBC News,2021-11-21 01:17:25+00:00,Covid: Fresh protests in Europe against new re...,https://www.facebook.com/bbcnews,He said this was because Germany was dealing w...


In [9]:
model_df.head(5)

,text,title
0,The C.D.C. endorsed Pfizer-BioNTech and Modern...,Austria Announces Lockdown and Vaccine Mandate
1,"Latvia, where 59% of the population are fully ...",Covid: Austria orders nationwide lockdown for ...
2,Please update your browser to see full interac...,Covid: Austrians heading towards lockdown for ...
3,The facility is closed temporarily as a New Yo...,Austria Enters Lockdown Amid Growing Resistance
4,He said this was because Germany was dealing w...,Covid: Fresh protests in Europe against new re...


In [10]:
preds = procDataStance(model_df, helper_path)

agree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[3]]
disagree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[2]]
discuss = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[0]]
unrelated = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[1]]

print(f"Agree: \n{model_df['title'].iloc[agree]}\n")
print(f"Disgree: \n{model_df['title'].iloc[disagree]}\n")
print(f"Discuss: \n{model_df['title'].iloc[discuss]}\n")
print(f"Unrelated: \n{model_df['title'].iloc[unrelated]}\n")

print("Discarding articles with stance: Disagree and Unrelated")

2021-11-26 04:13:03.783549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-26 04:13:03.817649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-26 04:13:03.818579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-26 04:13:03.819370: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Agree: 
0        Austria Announces Lockdown and Vaccine Mandate
1     Covid: Austria orders nationwide lockdown for ...
2     Covid: Austrians heading towards lockdown for ...
5             Austria locks down unvaccinated - Reuters
7     Austria readies COVID-19 lockdown for the unva...
8     Austria back to full lockdown, vaccines to be ...
9     Austria orders non-vaccinated people into COVI...
10    Austria reimposes full lockdown, makes vaccina...
14    Austria begins full coronavirus lockdown; not ...
15    Austrian COVID-19 cases keep rising as provinc...
16    Austria enters hard-to-enforce COVID-19 lockdo...
17    Here's a look at how countries are cracking do...
18    Austria imposes full lockdown, Germany may fol...
Name: title, dtype: object

Disgree: 
4     Covid: Fresh protests in Europe against new re...
6     Austria brings back COVID-19 lockdown, this ti...
11    Euro slumps as Austria reimposes lockdown; dol...
12    Austria going back in full lockdown as cases r...
19

2021-11-26 04:13:11.411468: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


In [11]:
model_df = discardStanceArticle(model_df, preds)
main_df = discardStanceArticle(main_df, preds)

main_df

,url,website,date,title,author,text
0,https://www.nytimes.com/live/2021/11/19/world/...,New York Times,2021-11-19 09:59:02+00:00,Austria Announces Lockdown and Vaccine Mandate,The New York Times,The C.D.C. endorsed Pfizer-BioNTech and Modern...
1,https://www.bbc.co.uk/news/world-europe-59283128,BBC News,2021-11-14 16:17:58+00:00,Covid: Austria orders nationwide lockdown for ...,https://www.facebook.com/bbcnews,"Latvia, where 59% of the population are fully ..."
2,https://www.bbc.co.uk/news/world-europe-59245018,BBC News,2021-11-11 18:13:32+00:00,Covid: Austrians heading towards lockdown for ...,https://www.facebook.com/bbcnews,Please update your browser to see full interac...
3,https://www.nytimes.com/live/2021/11/22/world/...,New York Times,2021-11-22 10:11:48+00:00,Austria Enters Lockdown Amid Growing Resistance,The New York Times,The facility is closed temporarily as a New Yo...
4,https://www.reuters.com/video/watch/idPklH?now...,Reuters,2021-11-15 17:05:02+00:00,Austria locks down unvaccinated - Reuters,Reuters Editorial,"Server Encountered an ErrorOur apologies, the ..."
5,https://globalnews.ca/news/8369158/austria-unv...,Global News,2021-11-12 14:27:18+00:00,Austria readies COVID-19 lockdown for the unva...,Staff,Send this page to someone via emailAustria‘s g...
6,https://www.reuters.com/video/watch/idOVF49RASR,Reuters,2021-11-19 15:46:08+00:00,"Austria back to full lockdown, vaccines to be ...",Reuters Editorial,"HealthAustria back to full lockdown, vaccines ..."
7,https://www.reuters.com/business/healthcare-ph...,Reuters,2021-11-14 12:12:00+00:00,Austria orders non-vaccinated people into COVI...,Reuters,Summary Austria has one of W.Europe's lowest v...
8,https://www.reuters.com/world/europe/austria-r...,Reuters,2021-11-19 09:40:00+00:00,"Austria reimposes full lockdown, makes vaccina...",Reuters,People wait in front of a vaccination bus duri...
9,https://www.nytimes.com/live/2021/11/15/world/...,New York Times,2021-11-15 11:00:04+00:00,Flu Shots Lag in the U.S.,The New York Times,New York City encouraged everyone 18 and older...


## Step 5: Fake News Detection

In [12]:
preds = procDataFake(main_df, helper_path)

true = [count for count, pred in enumerate(preds) if pred[0] > pred[1]]
fake = [count for count, pred in enumerate(preds) if pred[0] < pred[1]]

print(f"True: \n{model_df['title'].iloc[true]}\n")
print(f"Fake: \n{model_df['title'].iloc[fake]}\n")

print("Discarding Fake Articles")

True: 
0        Austria Announces Lockdown and Vaccine Mandate
1     Covid: Austria orders nationwide lockdown for ...
2     Covid: Austrians heading towards lockdown for ...
3       Austria Enters Lockdown Amid Growing Resistance
4             Austria locks down unvaccinated - Reuters
6     Austria back to full lockdown, vaccines to be ...
7     Austria orders non-vaccinated people into COVI...
8     Austria reimposes full lockdown, makes vaccina...
9                             Flu Shots Lag in the U.S.
10    Austria begins full coronavirus lockdown; not ...
11    Austrian COVID-19 cases keep rising as provinc...
12    Austria enters hard-to-enforce COVID-19 lockdo...
13    Here's a look at how countries are cracking do...
14    Austria imposes full lockdown, Germany may fol...
Name: title, dtype: object

Fake: 
5    Austria readies COVID-19 lockdown for the unva...
Name: title, dtype: object

Discarding Fake Articles


In [13]:
model_df = discardFakeArticle(model_df, preds)
main_df = discardFakeArticle(main_df, preds)

main_df

,url,website,date,title,author,text
0,https://www.nytimes.com/live/2021/11/19/world/...,New York Times,2021-11-19 09:59:02+00:00,austria announces lockdown vaccine mandate,The New York Times,cdc endorsed pfizerbiontech moderna booster sh...
1,https://www.bbc.co.uk/news/world-europe-59283128,BBC News,2021-11-14 16:17:58+00:00,covid austria orders nationwide lockdown unvac...,https://www.facebook.com/bbcnews,latvia 59 population fully vaccinated reimpose...
2,https://www.bbc.co.uk/news/world-europe-59245018,BBC News,2021-11-11 18:13:32+00:00,covid austrians heading towards lockdown unvac...,https://www.facebook.com/bbcnews,please update browser see full interactiveshow...
3,https://www.nytimes.com/live/2021/11/22/world/...,New York Times,2021-11-22 10:11:48+00:00,austria enters lockdown amid growing resistance,The New York Times,facility closed temporarily new york vaccinati...
4,https://www.reuters.com/video/watch/idPklH?now...,Reuters,2021-11-15 17:05:02+00:00,austria locks unvaccinated reuters,Reuters Editorial,server encountered errorour apologies content ...
5,https://www.reuters.com/video/watch/idOVF49RASR,Reuters,2021-11-19 15:46:08+00:00,austria back full lockdown vaccines compulsory...,Reuters Editorial,healthaustria back full lockdown vaccines comp...
6,https://www.reuters.com/business/healthcare-ph...,Reuters,2021-11-14 12:12:00+00:00,austria orders nonvaccinated people covid19 lo...,Reuters,summary austria one weuropes lowest vaccinatio...
7,https://www.reuters.com/world/europe/austria-r...,Reuters,2021-11-19 09:40:00+00:00,austria reimposes full lockdown makes vaccinat...,Reuters,people wait front vaccination bus coronavirus ...
8,https://www.nytimes.com/live/2021/11/15/world/...,New York Times,2021-11-15 11:00:04+00:00,flu shots lag us,The New York Times,new york city encouraged everyone 18 older get...
9,https://www.cp24.com/world/facing-covid-19-sur...,CP24 Toronto's Breaking News,2021-11-19 10:30:00+00:00,austria begins full coronavirus lockdown unvac...,Emily Schultheis And Kirsten Grieshaber,emily schultheis kirsten grieshaber associated...


## Step 6: Stance Detection for Similarity Matching

In [14]:
sim_match_df = model_df.loc[:, ["text"]]
sim_match_df["title"] = [fact_query] * len(sim_match_df.index)

sim_match_df

,text,title
0,The C.D.C. endorsed Pfizer-BioNTech and Modern...,There is a lockdown in austria
1,"Latvia, where 59% of the population are fully ...",There is a lockdown in austria
2,Please update your browser to see full interac...,There is a lockdown in austria
3,The facility is closed temporarily as a New Yo...,There is a lockdown in austria
4,"Server Encountered an ErrorOur apologies, the ...",There is a lockdown in austria
5,"HealthAustria back to full lockdown, vaccines ...",There is a lockdown in austria
6,Summary Austria has one of W.Europe's lowest v...,There is a lockdown in austria
7,People wait in front of a vaccination bus duri...,There is a lockdown in austria
8,New York City encouraged everyone 18 and older...,There is a lockdown in austria
9,"Emily Schultheis And Kirsten Grieshaber , The ...",There is a lockdown in austria


In [15]:
preds = procDataStance(model_df, helper_path)

agree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[3]]
disagree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[2]]
discuss = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[0]]
unrelated = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[1]]

print(f"Agree: \n{model_df['title'].iloc[agree]}\n")
print(f"Disgree: \n{model_df['title'].iloc[disagree]}\n")
print(f"Discuss: \n{model_df['title'].iloc[discuss]}\n")
print(f"Unrelated: \n{model_df['title'].iloc[unrelated]}\n")

Agree: 
0        Austria Announces Lockdown and Vaccine Mandate
1     Covid: Austria orders nationwide lockdown for ...
2     Covid: Austrians heading towards lockdown for ...
3       Austria Enters Lockdown Amid Growing Resistance
4             Austria locks down unvaccinated - Reuters
7     Austria reimposes full lockdown, makes vaccina...
9     Austria begins full coronavirus lockdown; not ...
11    Austria enters hard-to-enforce COVID-19 lockdo...
13    Austria imposes full lockdown, Germany may fol...
Name: title, dtype: object

Disgree: 
5     Austria back to full lockdown, vaccines to be ...
6     Austria orders non-vaccinated people into COVI...
8                             Flu Shots Lag in the U.S.
10    Austrian COVID-19 cases keep rising as provinc...
Name: title, dtype: object

Discuss: 
12    Here's a look at how countries are cracking do...
Name: title, dtype: object

Unrelated: 
Series([], Name: title, dtype: object)



## Step 7: Final Results

In [16]:
conf_score, agree_idx, disagree_idx = calcResult(main_df, preds)

print(f"\nThe fact '{fact_query}' can be said to be true with {round(conf_score, 4)*100}% confidence.")


The fact 'There is a lockdown in austria' can be said to be true with 71.43% confidence.


In [17]:
agree_df = main_df.iloc[agree_idx]

save_path = os.path.join(helper_path, "data/operational/agree.json")
agree_df.to_json(save_path, orient="records", force_ascii=False, indent=4)

agree_df

,url,website,date,title,author,text
0,https://www.nytimes.com/live/2021/11/19/world/...,New York Times,2021-11-19 09:59:02+00:00,austria announces lockdown vaccine mandate,The New York Times,cdc endorsed pfizerbiontech moderna booster sh...
1,https://www.bbc.co.uk/news/world-europe-59283128,BBC News,2021-11-14 16:17:58+00:00,covid austria orders nationwide lockdown unvac...,https://www.facebook.com/bbcnews,latvia 59 population fully vaccinated reimpose...
2,https://www.bbc.co.uk/news/world-europe-59245018,BBC News,2021-11-11 18:13:32+00:00,covid austrians heading towards lockdown unvac...,https://www.facebook.com/bbcnews,please update browser see full interactiveshow...
3,https://www.nytimes.com/live/2021/11/22/world/...,New York Times,2021-11-22 10:11:48+00:00,austria enters lockdown amid growing resistance,The New York Times,facility closed temporarily new york vaccinati...
4,https://www.reuters.com/video/watch/idPklH?now...,Reuters,2021-11-15 17:05:02+00:00,austria locks unvaccinated reuters,Reuters Editorial,server encountered errorour apologies content ...
7,https://www.reuters.com/world/europe/austria-r...,Reuters,2021-11-19 09:40:00+00:00,austria reimposes full lockdown makes vaccinat...,Reuters,people wait front vaccination bus coronavirus ...
9,https://www.cp24.com/world/facing-covid-19-sur...,CP24 Toronto's Breaking News,2021-11-19 10:30:00+00:00,austria begins full coronavirus lockdown unvac...,Emily Schultheis And Kirsten Grieshaber,emily schultheis kirsten grieshaber associated...
11,https://www.reuters.com/business/healthcare-ph...,Reuters,2021-11-15 09:19:00+00:00,austria enters hardtoenforce covid19 lockdown ...,Reuters,people walk past vaccination point amidst coro...
12,https://nationalpost.com/news/world/heres-a-lo...,National Post,2021-11-12 15:37:14+00:00,heres look countries cracking unvaccinated nat...,Bloomberg News,advertisement loaded yet article continues bel...
13,https://www.reuters.com/markets/europe/austria...,Reuters,2021-11-19 11:57:00+00:00,austria imposes full lockdown germany may foll...,Reuters,summary austria reimpose full lockdown mondayg...


In [18]:
disagree_df = main_df.iloc[disagree_idx]

save_path = os.path.join(helper_path, "data/operational/disagree.json")
disagree_df.to_json(save_path, orient="records", force_ascii=False, indent=4)

disagree_df

,url,website,date,title,author,text
5,https://www.reuters.com/video/watch/idOVF49RASR,Reuters,2021-11-19 15:46:08+00:00,austria back full lockdown vaccines compulsory...,Reuters Editorial,healthaustria back full lockdown vaccines comp...
6,https://www.reuters.com/business/healthcare-ph...,Reuters,2021-11-14 12:12:00+00:00,austria orders nonvaccinated people covid19 lo...,Reuters,summary austria one weuropes lowest vaccinatio...
8,https://www.nytimes.com/live/2021/11/15/world/...,New York Times,2021-11-15 11:00:04+00:00,flu shots lag us,The New York Times,new york city encouraged everyone 18 older get...
10,https://www.reuters.com/world/europe/austrian-...,Reuters,2021-11-18 10:27:00+00:00,austrian covid19 cases keep rising provinces p...,Reuters,pedestrians walk along shopping street austria...


In [19]:
print("Program Execution Complete. Please check agree.json and disagree.json for news articles")

Program Execution Complete. Please check agree.json and disagree.json for news articles
